# Dirichlet Laplace Indirect Method

**keys**: homogeneous Dirichlet bvp, single layer potential, ACA

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

We consider the Dirichlet boundary value problem 

$$ \left\{ \begin{array}{rcl l} \Delta u &=& 0, \quad &\Omega \subset \mathbb R^3\,,\\ \gamma_0 u&=& u_0, \quad &\Gamma = \partial \Omega\,.\end{array} \right. $$ 

Let us choose the following ansatz for the solution $u\in H^1(\Omega)$ (indirect ansatz) 

$$ u(x) = \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, j(y)\, \mathrm{d}\sigma_y }_{\displaystyle{ \mathrm{SL}(j) } }$$ 

and solve for the density $j\in H^{-\frac12}(\Gamma)$ by the boundary element method, i.e. the numerical solution of the variational formulation 

$$ \forall \, v\in H^{-\frac12}(\Gamma): \quad \left\langle \gamma_0 \left(\mathrm{SL}(j)\right), v \right\rangle_{-\frac12} = \left\langle u_0, v\right\rangle_{-\frac12} \,. $$
 

Define the geometry $\Omega \subset \mathbb R^3$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.3)).Curve(4)
Draw (mesh);

Create test and trial function finite element spaces for $H^{-\frac12}(\Gamma)$ according to the given mesh:  

In [ ]:
fesL2 = SurfaceL2(mesh, order=4)
u,v = fesL2.TnT()

Define Dirichlet data $u_0$ and compute the right hand side vector:

In [ ]:
u0 = 1/ sqrt( (x-1)**2 + (y-1)**2 + (z-1)**2 )
rhs = LinearForm (u0*v*ds(bonus_intorder=0)).Assemble()

The discretisation of the above variational formulation leads to a system of linear equations, ie 

$$ \mathrm{V} \, \mathrm{j} =  \mathrm{rhs} \,, $$ 
where $\mathrm{V}$ is the single layer potential operator. $\mathrm V$ is regular and symmetric.

Build the single layer operator $V$ as linear operator and solve for unknwon density $j$:

In [ ]:
j = GridFunction(fesL2)
pre = BilinearForm(u*v*ds, diagonal=True).Assemble().mat.Inverse()
with TaskManager():
    V = LaplaceSL(u*ds)*v*ds
    CG(mat = V.mat, pre=pre, rhs = rhs.vec, sol=j.vec, tol=1e-8, maxsteps=200, initialize=False, printrates=True)

In [ ]:
Draw (j, order=3);

## Evaluate solution at plane

In [ ]:
vismesh = (WorkPlane().RectangleC(4,4).Face()*Sphere((0,0,0),1)).GenerateMesh(maxh=0.2).Curve(4)
sol = GridFunction(H1(vismesh,order=3))

# SL = (LaplaceSL(u*ds)*v*ds).GetPotential(j, nearfield_experimental=False)
SL = LaplaceSL(u*ds(bonus_intorder=4))(j)

sol.Set (SL, definedon=vismesh.Boundaries(".*"))
Draw (sol, vismesh, order=3);

and compute error:

In [ ]:
norm_err = sqrt (Integrate ( (sol-u0)**2 * ds, vismesh))
norm_u0 = sqrt (Integrate ( (u0)**2 * ds, vismesh))
print ("||err|| =", norm_err, ", ||u0|| =", norm_u0)